# Baseline Model 

- Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)

In [1]:
%load_ext watermark

In [43]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [18]:
from tqdm import tqdm

from glob import glob

# import matplotlib.pyplot as plt
# %matplotlib inline
# from matplotlib import rcParams
# from cycler import cycler

# rcParams['figure.figsize'] = 12, 8 # 18, 5
# rcParams['axes.spines.top'] = False
# rcParams['axes.spines.right'] = False
# rcParams['axes.grid'] = True
# rcParams['axes.prop_cycle'] = cycler(color=['#365977'])
# rcParams['lines.linewidth'] = 2.5

# import seaborn as sns
# sns.set_theme()

# pd.set_option("max_columns", None)
# pd.set_option("max_rows", None)
# pd.set_option('display.max_colwidth', None)

from IPython.display import Markdown, display
def md(arg):
    display(Markdown(arg))

# from pandas_profiling import ProfileReport
# #report = ProfileReport(#DataFrame here#, minimal=True)
# #report.to

# import pyarrow.parquet as pq
# #df = pq.ParquetDataset(path_to_folder_with_parquets, filesystem=None).read_pandas().to_pandas()

# import json
# def open_file_json(path,mode='r',var=None):
#     if mode == 'w':
#         with open(path,'w') as f:
#             json.dump(var, f)
#     if mode == 'r':
#         with open(path,'r') as f:
#             return json.load(f)

# import functools
# import operator
# def flat(a):
#     return functools.reduce(operator.iconcat, a, [])

# import json
# from glob import glob
# from typing import NewType


# DictsPathType = NewType("DictsPath", str)


# def open_file_json(path):
#     with open(path, "r") as f:
#         return json.load(f)

# class LoadDicts:
#     def __init__(self, dict_path: DictsPathType = "./data"):
#         Dicts_glob = glob(f"{dict_path}/*.json")
#         self.List = []
#         self.Dict = {}
#         for path_json in Dicts_glob:
#             name = path_json.split("/")[-1].replace(".json", "")
#             self.List.append(name)
#             self.Dict[name] = open_file_json(path_json)
#             setattr(self, name, self.Dict[name])


In [19]:
# Run this cell before close.
%watermark -d --iversion -b -r -g -m -v
!cat /proc/cpuinfo |grep 'model name'|head -n 1 |sed -e 's/model\ name/CPU/'
!free -h |cut -d'i' -f1  |grep -v total

Python implementation: CPython
Python version       : 3.9.6
IPython version      : 7.25.0

Compiler    : GCC 8.3.0
OS          : Linux
Release     : 5.11.0-7620-generic
Machine     : x86_64
Processor   : 
CPU cores   : 4
Architecture: 64bit

Git hash: c918b8f2b7a6dfa5a6f9edd872f7a0bdaf5000ef

Git repo: https://github.com/ysraell/creditcardfraud.git

Git branch: main

numpy : 1.19.5
pandas: 1.3.1

CPU	: Intel(R) Core(TM) i7-7500U CPU @ 2.70GHz
Mem:           15G
Swap:         4.0G


In [49]:
#
N_fraud_test = 200
N_truth_test = int(1e5)

#
split_seeds = [13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53]

# random_state used by RandomForestClassifier
random_state = 42

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [67]:
for ds_path in glob('/work/data/creditcard*.csv'):
    df = pd.read_csv(ds_path)
    target_col = 'Class'
    ds_cols = [col for col in df.columns if 'V' in col]+['Amount']
    df = df[ds_cols+[target_col]]
    df_fraud = df.query('Class == 1').reset_index(drop=True).copy()
    df_truth = df.query('Class == 0').reset_index(drop=True).copy()
    del df
    for seed in split_seeds:
        df_fraud_train, df_fraud_test = train_test_split(df_fraud, test_size=N_fraud_test, random_state=seed)
        df_truth_train, df_truth_test = train_test_split(df_truth, test_size=N_truth_test, random_state=seed)
        df_train = pd.concat([df_fraud_train, df_truth_train]).reset_index(drop=True)
        df_test = pd.concat([df_fraud_test, df_truth_test]).reset_index(drop=True)
        rf = RandomForestClassifier(random_state=random_state)
        break
    break

In [68]:
rf.fit(df_train[ds_cols].to_numpy(), df_train[target_col].to_numpy())

RandomForestClassifier(random_state=42)

In [66]:
df_train

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-2.812318,5.798184,-3.937366,4.908679,-7.804212,-2.282616,-11.832920,11.998343,-9.294708,-12.505578,...,4.042524,0.677383,-0.844854,1.293193,0.520532,0.637490,-3.454365,-2.056070,0.76,1
1,-2.836502,6.199016,-4.045557,5.138158,-9.075513,-2.314132,-13.111609,14.861249,-9.007673,-12.748092,...,4.253680,0.456343,-1.880740,1.421024,-0.404037,0.375075,-5.958685,-3.717119,106.55,1
2,-0.132144,1.356913,-0.423550,3.289147,0.944220,0.227788,0.179644,0.020962,-1.283723,-0.037241,...,-0.050276,0.263549,-0.284622,1.413553,-0.459013,0.998133,-0.403269,0.204279,0.76,1
3,-1.290881,1.520135,-2.025208,0.140952,-0.135347,0.235416,-2.271613,-2.452063,-0.491836,-4.197820,...,-0.097409,0.768524,-0.132557,-0.791253,0.667949,-1.116858,0.356037,0.086113,50.00,1
4,0.639064,2.275106,-2.914774,3.357290,1.301779,-1.323806,-0.753634,0.248073,-3.081087,-3.638179,...,0.179061,-1.106535,-0.131013,-0.133476,0.711474,0.213202,1.219162,1.002689,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184602,1.320280,-0.121647,-1.385145,0.892323,0.521532,-0.347084,0.457182,-0.272000,0.415006,0.246182,...,0.052492,0.087820,-0.113515,0.337043,0.895254,-1.056006,-0.109519,-0.153937,74.00,0
184603,-0.714904,0.865890,-0.390835,0.796321,0.689419,-0.769941,0.807519,-0.038251,-0.874291,-0.119013,...,0.252961,1.028295,-0.651090,0.027994,0.078615,-0.801479,0.645435,0.251633,13.15,0
184604,1.381446,0.080942,-1.518457,0.745023,0.865977,-0.412972,0.692008,-0.455975,-0.166580,0.295795,...,0.162422,0.422203,-0.201972,-0.008807,1.106101,-1.013209,-0.110389,-0.157391,64.02,0
184605,1.434087,0.083291,-1.185812,0.350559,0.336638,-0.657810,0.204577,-0.331196,0.350810,-0.369933,...,-0.474129,-1.197167,0.535954,1.012188,-0.481882,0.415136,-0.149793,-0.079309,8.13,0


In [53]:
rf.fit(df_fraud_train.to_numpy(), 

Signature: rf.fit(X, y, sample_weight=None)
Docstring:
Build a forest of trees from the training set (X, y).

Parameters
----------
X : {array-like, sparse matrix} of shape (n_samples, n_features)
    The training input samples. Internally, its dtype will be converted
    to ``dtype=np.float32``. If a sparse matrix is provided, it will be
    converted into a sparse ``csc_matrix``.

y : array-like of shape (n_samples,) or (n_samples, n_outputs)
    The target values (class labels in classification, real numbers in
    regression).

sample_weight : array-like of shape (n_samples,), default=None
    Sample weights. If None, then samples are equally weighted. Splits
    that would create child nodes with net zero or negative weight are
    ignored while searching for a split in each node. In the case of
    classification, splits are also ignored if they would result in any
    single class carrying a negative weight in either child node.

Returns
-------
self : object
File:      /usr/loc